# Segmentacion de Uso de Suelo en San Juan de Pasto

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeos/geoai/blob/main/GeoIA-Pasto/notebooks/05_Segmentacion_Uso_Suelo_Pasto.ipynb)

---

## Objetivos

1. Clasificar el uso del suelo en Pasto usando modelos pre-entrenados
2. Identificar areas urbanas, agricolas, bosques y cuerpos de agua
3. Crear mapas tematicos de cobertura del suelo
4. Entrenar un modelo personalizado para la region

## 1. Instalacion de Dependencias

In [ ]:
%pip install -q geoai-py leafmap pystac-client planetary-computer rioxarray torch torchvision segmentation-models-pytorch

## 2. Importacion de Librerias

In [ ]:
import geoai
import leafmap
import rioxarray as rxr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import geopandas as gpd
import pystac_client
import planetary_computer
import torch
import os
import warnings

warnings.filterwarnings("ignore")

print(f"GeoAI version: {geoai.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA disponible: {torch.cuda.is_available()}")

## 3. Definicion del Area de Estudio y Clases

In [ ]:
# Coordenadas de San Juan de Pasto
PASTO_CENTER = {"lat": 1.2136, "lon": -77.2811}
PASTO_BBOX = (-77.35, 1.15, -77.20, 1.30)

# Clases de uso del suelo para Pasto
CLASES_USO_SUELO = {
    0: {"nombre": "Sin datos", "color": "#000000"},
    1: {"nombre": "Agua", "color": "#0077be"},
    2: {"nombre": "Urbano", "color": "#ff6b6b"},
    3: {"nombre": "Bosque", "color": "#228b22"},
    4: {"nombre": "Cultivos", "color": "#ffd700"},
    5: {"nombre": "Pastos", "color": "#90ee90"},
    6: {"nombre": "Suelo desnudo", "color": "#d2691e"},
    7: {"nombre": "Paramo", "color": "#dda0dd"},
    8: {"nombre": "Nubes", "color": "#ffffff"},
}

print("Clases de uso del suelo para Pasto:")
for id_clase, info in CLASES_USO_SUELO.items():
    print(f"  {id_clase}: {info['nombre']}")

## 4. Descargar Imagen Sentinel-2

In [ ]:
# Conectar a Planetary Computer
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

# Buscar imagen con pocas nubes
search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=PASTO_BBOX,
    datetime="2024-01-01/2024-12-31",
    query={"eo:cloud_cover": {"lt": 10}},
)

items = list(search.items())
print(f"Imagenes encontradas: {len(items)}")

if items:
    best_item = min(items, key=lambda x: x.properties.get("eo:cloud_cover", 100))
    print(f"\nImagen seleccionada: {best_item.id}")
    print(f"Fecha: {best_item.datetime}")
    print(f"Nubes: {best_item.properties.get('eo:cloud_cover')}%")

In [ ]:
# Cargar bandas necesarias
if items:
    bandas = [
        "B02",
        "B03",
        "B04",
        "B08",
        "B11",
        "B12",
    ]  # Blue, Green, Red, NIR, SWIR1, SWIR2

    datos = {}
    print("Cargando bandas...")
    for banda in bandas:
        if banda in best_item.assets:
            url = best_item.assets[banda].href
            da = rxr.open_rasterio(url).squeeze()
            da = da.rio.clip_box(*PASTO_BBOX)
            datos[banda] = da
            print(f"  {banda}: {da.shape}")

    print("\nBandas cargadas exitosamente")

## 5. Crear Composiciones de Color

In [ ]:
def normalizar_banda(banda, min_val=0, max_val=3000):
    """Normaliza una banda al rango [0, 1]"""
    arr = banda.values.astype(float)
    arr = np.clip(arr, min_val, max_val)
    return (arr - min_val) / (max_val - min_val)


# Crear composiciones
if datos:
    # Color verdadero (RGB)
    rgb = np.dstack(
        [
            normalizar_banda(datos["B04"]),
            normalizar_banda(datos["B03"]),
            normalizar_banda(datos["B02"]),
        ]
    )

    # Falso color (NIR-R-G) - Vegetacion en rojo
    falso_color = np.dstack(
        [
            normalizar_banda(datos["B08"]),
            normalizar_banda(datos["B04"]),
            normalizar_banda(datos["B03"]),
        ]
    )

    # SWIR (para areas urbanas y suelo)
    swir = np.dstack(
        [
            normalizar_banda(datos["B12"]),
            normalizar_banda(datos["B08"]),
            normalizar_banda(datos["B04"]),
        ]
    )

    print("Composiciones creadas")

In [ ]:
# Visualizar composiciones
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(rgb)
axes[0].set_title("Color Verdadero (RGB)", fontsize=12)
axes[0].axis("off")

axes[1].imshow(falso_color)
axes[1].set_title("Falso Color (NIR-R-G)\nVegetacion en rojo", fontsize=12)
axes[1].axis("off")

axes[2].imshow(swir)
axes[2].set_title("SWIR Compuesto\nAreas urbanas en cyan", fontsize=12)
axes[2].axis("off")

plt.suptitle("Composiciones de Color - San Juan de Pasto", fontsize=14)
plt.tight_layout()
plt.savefig("./data/composiciones_pasto.png", dpi=150, bbox_inches="tight")
plt.show()

## 6. Clasificacion Basada en Indices

In [ ]:
# Calcular indices espectrales
if datos:
    nir = datos["B08"].values.astype(float)
    red = datos["B04"].values.astype(float)
    green = datos["B03"].values.astype(float)
    blue = datos["B02"].values.astype(float)
    swir1 = datos["B11"].values.astype(float)
    swir2 = datos["B12"].values.astype(float)

    # NDVI - Vegetacion
    ndvi = (nir - red) / (nir + red + 1e-10)

    # NDWI - Agua
    ndwi = (green - nir) / (green + nir + 1e-10)

    # NDBI - Areas construidas
    ndbi = (swir1 - nir) / (swir1 + nir + 1e-10)

    # BSI - Suelo desnudo
    bsi = ((swir1 + red) - (nir + blue)) / ((swir1 + red) + (nir + blue) + 1e-10)

    print("Indices calculados:")
    print(f"  NDVI (Vegetacion): min={ndvi.min():.2f}, max={ndvi.max():.2f}")
    print(f"  NDWI (Agua): min={ndwi.min():.2f}, max={ndwi.max():.2f}")
    print(f"  NDBI (Construido): min={ndbi.min():.2f}, max={ndbi.max():.2f}")
    print(f"  BSI (Suelo): min={bsi.min():.2f}, max={bsi.max():.2f}")

In [ ]:
# Visualizar indices
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

im1 = axes[0, 0].imshow(ndvi, cmap="RdYlGn", vmin=-0.2, vmax=0.8)
axes[0, 0].set_title("NDVI - Vegetacion", fontsize=12)
axes[0, 0].axis("off")
plt.colorbar(im1, ax=axes[0, 0], fraction=0.046)

im2 = axes[0, 1].imshow(ndwi, cmap="Blues", vmin=-0.3, vmax=0.3)
axes[0, 1].set_title("NDWI - Agua", fontsize=12)
axes[0, 1].axis("off")
plt.colorbar(im2, ax=axes[0, 1], fraction=0.046)

im3 = axes[1, 0].imshow(ndbi, cmap="Reds", vmin=-0.3, vmax=0.3)
axes[1, 0].set_title("NDBI - Areas Construidas", fontsize=12)
axes[1, 0].axis("off")
plt.colorbar(im3, ax=axes[1, 0], fraction=0.046)

im4 = axes[1, 1].imshow(bsi, cmap="YlOrBr", vmin=-0.3, vmax=0.3)
axes[1, 1].set_title("BSI - Suelo Desnudo", fontsize=12)
axes[1, 1].axis("off")
plt.colorbar(im4, ax=axes[1, 1], fraction=0.046)

plt.suptitle("Indices Espectrales - San Juan de Pasto", fontsize=14)
plt.tight_layout()
plt.savefig("./data/indices_espectrales_pasto.png", dpi=150, bbox_inches="tight")
plt.show()

## 7. Clasificacion Simple por Umbrales

In [ ]:
# Clasificacion basada en umbrales
def clasificar_uso_suelo(ndvi, ndwi, ndbi, bsi):
    """
    Clasifica el uso del suelo basado en indices espectrales.
    """
    clasificacion = np.zeros_like(ndvi, dtype=np.uint8)

    # 1: Agua (NDWI alto)
    clasificacion[ndwi > 0.1] = 1

    # 3: Bosque (NDVI muy alto)
    clasificacion[(ndvi > 0.5) & (clasificacion == 0)] = 3

    # 5: Pastos (NDVI moderado)
    clasificacion[(ndvi > 0.3) & (ndvi <= 0.5) & (clasificacion == 0)] = 5

    # 4: Cultivos (NDVI bajo-moderado)
    clasificacion[(ndvi > 0.2) & (ndvi <= 0.3) & (clasificacion == 0)] = 4

    # 2: Urbano (NDBI alto, NDVI bajo)
    clasificacion[(ndbi > 0) & (ndvi < 0.2) & (clasificacion == 0)] = 2

    # 6: Suelo desnudo (BSI alto)
    clasificacion[(bsi > 0.1) & (clasificacion == 0)] = 6

    # 7: Paramo (alta altitud, vegetacion especial)
    # Esto requeriria datos de elevacion adicionales

    return clasificacion


# Clasificar
clasificacion = clasificar_uso_suelo(ndvi, ndwi, ndbi, bsi)

# Contar pixeles por clase
print("Distribucion de uso del suelo:")
print("=" * 40)
total = clasificacion.size
for id_clase, info in CLASES_USO_SUELO.items():
    count = np.sum(clasificacion == id_clase)
    pct = (count / total) * 100
    if pct > 0:
        print(f"{info['nombre']:20s}: {pct:6.2f}%")

In [ ]:
# Visualizar clasificacion
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Imagen RGB
axes[0].imshow(rgb)
axes[0].set_title("Imagen Sentinel-2 (RGB)", fontsize=12)
axes[0].axis("off")

# Clasificacion
colors = [info["color"] for info in CLASES_USO_SUELO.values()]
cmap = mcolors.ListedColormap(colors)
bounds = list(range(len(CLASES_USO_SUELO) + 1))
norm = mcolors.BoundaryNorm(bounds, cmap.N)

im = axes[1].imshow(clasificacion, cmap=cmap, norm=norm)
axes[1].set_title("Clasificacion de Uso del Suelo", fontsize=12)
axes[1].axis("off")

# Leyenda
labels = [info["nombre"] for info in CLASES_USO_SUELO.values()]
cbar = plt.colorbar(im, ax=axes[1], fraction=0.046, ticks=range(len(labels)))
cbar.ax.set_yticklabels(labels)

plt.suptitle("Uso del Suelo - San Juan de Pasto", fontsize=14)
plt.tight_layout()
plt.savefig("./data/uso_suelo_pasto.png", dpi=150, bbox_inches="tight")
plt.show()

## 8. Segmentacion con Modelos Pre-entrenados

In [ ]:
# Preparar datos para modelo de segmentacion
# GeoAI incluye funcionalidades para segmentacion semantica

print("Para segmentacion avanzada con modelos pre-entrenados:")
print("")
print("1. Usar geoai.train() para entrenar modelos personalizados")
print("2. Usar geoai.segment() para inferencia con modelos existentes")
print("3. Arquitecturas disponibles: U-Net, DeepLabV3+, FPN, etc.")
print("")
print("Ejemplo de uso:")
print("")
print("# Entrenar modelo")
print("geoai.train(")
print("    images_dir='./data/chips/',")
print("    labels_dir='./data/labels/',")
print("    output_dir='./models/',")
print("    architecture='unet',")
print("    encoder='resnet50',")
print("    epochs=50")
print(")")

## 9. Guardar Resultados

In [ ]:
# Crear directorio
os.makedirs("./data", exist_ok=True)

# Guardar clasificacion como GeoTIFF
if datos:
    # Usar la referencia espacial de una de las bandas
    clasificacion_da = datos["B04"].copy()
    clasificacion_da.values = clasificacion
    clasificacion_da.name = "uso_suelo"
    clasificacion_da.rio.to_raster("./data/uso_suelo_pasto.tif")
    print("Guardado: uso_suelo_pasto.tif")

    # Guardar indices
    ndvi_da = datos["B04"].copy()
    ndvi_da.values = ndvi
    ndvi_da.rio.to_raster("./data/ndvi_pasto_2.tif")
    print("Guardado: ndvi_pasto_2.tif")

## 10. Mapa Interactivo

In [ ]:
# Crear mapa interactivo
m = leafmap.Map(
    center=[PASTO_CENTER["lat"], PASTO_CENTER["lon"]], zoom=12, height="600px"
)

m.add_basemap("Esri.WorldImagery")
m.add_basemap("OpenStreetMap")

# Agregar clasificacion si existe
uso_suelo_file = "./data/uso_suelo_pasto.tif"
if os.path.exists(uso_suelo_file):
    m.add_raster(uso_suelo_file, layer_name="Uso del Suelo", colormap="tab10")

m.add_layer_control()
m

## 11. Resumen

En este notebook hemos aprendido a:

1. **Crear composiciones de color** para visualizar diferentes caracteristicas
2. **Calcular indices espectrales** (NDVI, NDWI, NDBI, BSI)
3. **Clasificar uso del suelo** usando umbrales
4. **Identificar clases** relevantes para Pasto (urbano, bosque, cultivos, agua)
5. **Guardar resultados** como GeoTIFF georreferenciados

### Clases identificadas en Pasto:

- **Urbano**: Centro de la ciudad, zonas residenciales
- **Bosque**: Laderas del Galeras, areas de reserva
- **Cultivos**: Valle de Atriz, zonas agricolas
- **Pastos**: Areas ganaderas

### Proximos pasos:

- Entrenar modelos de deep learning para mejor precision
- Validar resultados con datos de campo
- Analizar cambios temporales en el uso del suelo